In [72]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import warnings
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from statsmodels.tsa.stattools import pacf
from tqdm import tqdm
warnings.filterwarnings("ignore")
close = pd.read_parquet("../research_IC/Adj_close.parquet")
df_close = close
df_close.dropna(how='all', axis=0, inplace=True)
df_close.dropna(how='all', axis=1, inplace=True)
pct_change = df_close.pct_change().dropna(axis=0, how='all').loc['2010-01-01':] # 3/19
pct_change = pct_change.resample("W").sum()

In [ ]:
def get_best_lags(df):
    # df = pct_change_train[ticker]
    pacf_values = pacf(df, method='ywm')
    N = len(df)
    threshold = 1.96 / np.sqrt(N)
    significant_indices = np.where(np.abs(pacf_values) > threshold)[0]
    # significant_pacf_values = pacf_values[significant_indices]
    if len(significant_indices) >= 2:
        best_lag_n = significant_indices[1]
    elif len(significant_indices) == 1:
        best_lag_n = np.argmax(np.abs(pacf_values[1:])) + 1
    return best_lag_n

stock_list = list(df_close.columns)
factors = {}
indices = []
backtest_period = 25
list_stock = list(pct_change.columns)
for index in tqdm(range(len(pct_change)-backtest_period)):
    prediction_factor = [0] * len(list_stock)

    end_date_of_calculation = pct_change.iloc[index+backtest_period].name
    start_date_of_calculation = pct_change.iloc[index].name
    pct_change_train = pct_change[start_date_of_calculation:end_date_of_calculation]
    for ticker in stock_list[:100]:
        #get max lags with pacf
        train_data = pct_change_train[ticker]
        if np.mean(train_data) == 0:
            appender = 0
        else:
            best_lag_n = get_best_lags(train_data)
            model = sm.tsa.arima.ARIMA(train_data, order=(best_lag_n, 0, 0)) # order = (ar_t, i_t, ma_t), for ar model only => ma_t = 0
            model_fit = model.fit()
            prediction = model_fit.forecast()
            appender = prediction.item()
        prediction_factor.append(appender)
    factors[end_date_of_calculation] = prediction_factor
factors_final = pd.DataFrame(factors).T
factors_final.columns = stock_list[:100]

In [94]:
factors_processed = factors_final[factors_final != 0].dropna(how='all', axis=1)
factors_processed.columns = stock_list[:100]
factors_final.to_parquet("ARIMA_freqW_factors_20d_bestlag_first100.parquet")


In [ ]:
# calculate ICIR